# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [73]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bluff,-46.6000,168.3333,12.21,67,48,12.35,NZ,1675829655
1,1,perth,-31.9333,115.8333,31.12,42,0,3.60,AU,1675829700
2,2,port alfred,-33.5906,26.8910,20.10,90,100,1.79,ZA,1675829600
3,3,vaini,-21.2000,-175.2000,25.09,100,75,2.57,TO,1675829589
4,4,butaritari,3.0707,172.7902,27.41,80,100,9.67,KI,1675829578


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

Instruction: Narrow down the city_data_df DataFrame to find your ideal weather condition. For example:
- A max temperature lower than 27 degrees but higher than 21
- Wind speed less than 4.5 m/s
- Zero cloudiness

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_citydata_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) & 
                                    (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0), :]

# Drop any rows with null values
narrow_citydata_df = narrow_citydata_df.dropna(how='any')

# Display sample data
narrow_citydata_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
59,59,mandla,22.6000,80.3833,22.43,26,0,2.68,IN,1675829983
152,152,brandon,27.9378,-82.2859,21.13,74,0,1.79,US,1675829881
318,318,salalah,17.0151,54.0924,21.05,56,0,1.03,OM,1675829873
331,331,reconquista,-29.1500,-59.6500,26.47,30,0,3.41,AR,1675830014
348,348,ixtapa,20.7000,-105.2000,21.24,83,0,0.00,MX,1675830060
402,402,santa maria,-29.6842,-53.8069,21.29,61,0,2.86,BR,1675830079
457,457,lazaro cardenas,17.9583,-102.2000,23.07,83,0,1.30,MX,1675830098
550,550,bull savanna,17.8868,-77.5902,21.39,82,0,3.15,JM,1675830122


### Step 3: Create a new DataFrame called `hotel_df`.

Instruction: For each city, use the Geoapify API to find the first hotel located within 10,000 meters of your coordinates.

Add the hotel name and the country as additional information in the hover message for each city on the map as in the following image:

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_citydata_df.loc[:,["City","Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
59,mandla,IN,22.6000,80.3833,26,
152,brandon,US,27.9378,-82.2859,74,
318,salalah,OM,17.0151,54.0924,56,
331,reconquista,AR,-29.1500,-59.6500,30,
348,ixtapa,MX,20.7000,-105.2000,83,
402,santa maria,BR,-29.6842,-53.8069,61,
457,lazaro cardenas,MX,17.9583,-102.2000,83,
550,bull savanna,JM,17.8868,-77.5902,82,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [88]:
# Set parameters to search for a hotel
radius = 10000
#params["location"] = str(hotel_df["Lat"]) + "," +str(hotel_df["Lng"])

params = {"categories": "accommodation.hotel",
          "keyword": "Hotel",
          "location" : str(hotel_df["Lat"]) + "," +str(hotel_df["Lng"]),
          "limit":limit,
          "api_keys":geoapify_key
}
type: "FeatureCollection"


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mandla - nearest hotel: No hotel found
brandon - nearest hotel: No hotel found
salalah - nearest hotel: No hotel found
reconquista - nearest hotel: No hotel found
ixtapa - nearest hotel: No hotel found
santa maria - nearest hotel: No hotel found
lazaro cardenas - nearest hotel: No hotel found
bull savanna - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
59,mandla,IN,22.6000,80.3833,26,No hotel found
152,brandon,US,27.9378,-82.2859,74,No hotel found
318,salalah,OM,17.0151,54.0924,56,No hotel found
331,reconquista,AR,-29.1500,-59.6500,30,No hotel found
348,ixtapa,MX,20.7000,-105.2000,83,No hotel found
402,santa maria,BR,-29.6842,-53.8069,61,No hotel found
457,lazaro cardenas,MX,17.9583,-102.2000,83,No hotel found
550,bull savanna,JM,17.8868,-77.5902,82,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [57]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    #"Hotel Name", 
    #"Country",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    size = "Humidity",
    c ="Country"
)

# Display the map
map_plot_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)